In [4]:
import sys
import pandas as pd
import os
import re
import glob
import shutil
import numpy as np
from pathlib import Path
import shutil
from sklearn.calibration import CalibratedClassifierCV

from IPython.core.display import display, HTML
# make the Jupyter notebook use the full screen width
display(HTML("<style>.container { width:99% !important; }</style>"))

In [2]:
#DATA_DIR= "/notebook/nas-trainings/arne/DGFISMA/DATA/doc_classifier/EUROLEX/Raw"
DATA_DIR= "/notebook/nas-trainings/arne/DGFISMA/DATA/doc_classifier/DATA_DUMP_28_04"

In [3]:
import glob
import json
Eurolex_files=glob.glob( os.path.join( DATA_DIR, "EURLEX" , "*jsonl" )  )

BIS_files=glob.glob( os.path.join( DATA_DIR, "BIS" , "*jsonl" )  )

EIOPA_files=glob.glob( os.path.join( DATA_DIR, "EIOPA" , "*jsonl" )  )

FSB_files=glob.glob( os.path.join( DATA_DIR, "FSB" , "*jsonl" )  )

SRB_files=glob.glob( os.path.join( DATA_DIR, "SRB" , "*jsonl" )  )


In [4]:
files=Eurolex_files + BIS_files + EIOPA_files + FSB_files + SRB_files
len(files)

17377

In [5]:
all_jsons=[]

for file in files:
    jsonl=open( file ).read().rstrip("\n").split( "\n" )
    json_file = [json.loads( json_line ) for json_line in jsonl]
    all_jsons.append( json_file )


In [6]:
len( all_jsons )

17377

In [7]:
status=[]
date=[]
title=[]
summary=[]

for json in all_jsons:
    metadata=json[0]
    if 'status' in metadata.keys():
        status.append( metadata['status'][0]   )
    else: 
        status.append( '-'  )
        
    if 'date' in metadata.keys():
        date.append( metadata['date'][0]   )
    else: 
        date.append( '-'  )
    
    if 'title' in metadata.keys():
        title.append( metadata['title'][0]   )
    else: 
        title.append( '-'  )
        
    if 'summary' in metadata.keys():
        summary.append( metadata['summary'][0]   )
    else: 
        summary.append( '-'  )

In [8]:
len(status)

17377

In [9]:
target=len(status)*['-']
for i,stat in enumerate( status):
    if ("No longer in force" in stat) or ( "Superseded" in stat ) or ( "Modified" in stat ):
        target[i]="declined"
    elif ("In force" in stat) or ("Current" in stat) or ("Consolidated" in stat):
        target[i]='accepted'
    elif ("Date of entry into force unknown" in stat ) or ('-' in stat):
        target[i]='unknown'
    else:
        target[i]='unknown'
                     
import collections
collections.Counter(target)

Counter({'declined': 9028, 'unknown': 5966, 'accepted': 2383})

In [10]:
titles_clean=[]
date_clean=[]
target_clean=[]
for tit, dat, tar in zip( title, date, target ):
    if (tar!='unknown') and (dat !='-') and (tit !='-'):
        titles_clean.append( tit )
        date_clean.append(dat)
        target_clean.append(  tar  )
        
year=[ int( one_date.split("-")[0] ) for one_date in date_clean ]
#year=np.array( year ).reshape( -1,1  )

In [11]:
titles_concat=[]
for yea,tit in zip(year,titles_clean):
    titles_concat.append( str( yea)+' '+ tit)

In [104]:
#s = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", s)
#print(s)

In [105]:
#for i,tit in enumerate(titles_concat):
#    titles_concat[i]=''.join([i for i in tit if not i.isdigit()])
    
#titles_concat

In [96]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
import scipy

vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                         stop_words='english'  )

vectorizer.fit(titles_concat)
X = vectorizer.transform( titles_concat )

#scaler = StandardScaler()
#scaler.fit( year  )
#year=scaler.transform( year  )

#concat=scipy.sparse.hstack(  ( X, year ) )

#check that the year is not thrown away in the vocab

In [97]:
y=[ 1 if item=='accepted' else 0 for item in target_clean  ]
y=np.array( y ).reshape(-1,1)


In [98]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=45)

In [99]:
from sklearn.svm import LinearSVC
classifier=LinearSVC(penalty='l1', loss='squared_hinge' , dual=False, class_weight='balanced')

calibrated_classifier = CalibratedClassifierCV(classifier , cv=5 ) 

calibrated_classifier.fit(  X_train, y_train.ravel()  )


CalibratedClassifierCV(base_estimator=LinearSVC(C=1.0, class_weight='balanced', dual=False, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l1', random_state=None, tol=0.0001,
     verbose=0),
            cv=5, method='sigmoid')

In [100]:
y_pred=calibrated_classifier.predict( X_test )

In [101]:
from sklearn.metrics import classification_report
print(classification_report(  y_test , y_pred   ))

              precision    recall  f1-score   support

           0       0.92      0.95      0.94      2431
           1       0.83      0.72      0.77       746

   micro avg       0.90      0.90      0.90      3177
   macro avg       0.87      0.84      0.85      3177
weighted avg       0.90      0.90      0.90      3177



In [97]:
from sklearn.metrics import classification_report
print(classification_report(  y_test , y_pred   ))

              precision    recall  f1-score   support

           0       0.92      0.96      0.94      2431
           1       0.84      0.72      0.77       746

   micro avg       0.90      0.90      0.90      3177
   macro avg       0.88      0.84      0.85      3177
weighted avg       0.90      0.90      0.90      3177



In [ ]:
#test BERT on this (concat date with the title and send to BERT...)

In [85]:
from sklearn.metrics import classification_report
print(classification_report(  y_test , y_pred   ))

              precision    recall  f1-score   support

           0       0.92      0.95      0.94      2431
           1       0.82      0.74      0.78       746

   micro avg       0.90      0.90      0.90      3177
   macro avg       0.87      0.84      0.86      3177
weighted avg       0.90      0.90      0.90      3177



In [39]:
for one_date, one_status, one_title in zip( date, status, title ):
    print(one_status)
    print( one_date, one_title  )

Superseded
1992-09-28T00:00:00Z A framework for measuring and managing liquidity 
-
2019-06-19T00:00:00Z Guidelines on non-significant benchmarks under the Benchmarks Regulation
-
2019-10-01T00:00:00Z Guidelines on risk factors under the Prospectus Regulation
-
2013-08-29T00:00:00Z Policy Framework for Strengthening Oversight and Regulation of Shadow Banking Entities
-
2020-02-04T00:00:00Z Guidelines on enforcement of financial information
In force
2016-12-14T00:00:00Z Decision (EU) 2016/2371 of the European Parliament and of the Council of 14 December 2016 providing further macro-financial assistance to the Hashemite Kingdom of Jordan
In force
2017-04-05T00:00:00Z Decision (EU) 2017/741 of the European Parliament and of the Council of 5 April 2017 on the mobilisation of the European Union Solidarity Fund to provide assistance to the United Kingdom, Cyprus and Portugal
-
2013-11-18T00:00:00Z Principles for an Effective Risk Appetite Framework
Superseded
2000-01-21T00:00:00Z Industry vi

In [93]:
year=[ float( one_date.split("-")[0] ) for one_date in date ]
year=np.array( year ).reshape( -1,1  )

In [94]:
title

['A framework for measuring and managing liquidity ',
 'Guidelines on non-significant benchmarks under the Benchmarks Regulation',
 'Guidelines on risk factors under the Prospectus Regulation',
 'Policy Framework for Strengthening Oversight and Regulation of Shadow Banking Entities',
 'Guidelines on enforcement of financial information',
 'Decision (EU) 2016/2371 of the European Parliament and of the Council of 14 December 2016 providing further macro-financial assistance to the Hashemite Kingdom of Jordan',
 'Decision (EU) 2017/741 of the European Parliament and of the Council of 5 April 2017 on the mobilisation of the European Union Solidarity Fund to provide assistance to the United Kingdom, Cyprus and Portugal',
 'Principles for an Effective Risk Appetite Framework',
 'Industry views on credit risk mitigation ',
 'Guidelines on standardised procedures and messaging protocols under Article 6(2) of Regulation (EU) No 909/2014',
 'Standards and Processes for Global Securities Financin

In [115]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                         stop_words='english'  )

vectorizer.fit(title)
X = vectorizer.transform( title )

scaler = StandardScaler()
scaler.fit( np.array( year ).reshape( -1,1  )  )
year=scaler.transform( year  )

concat=scipy.sparse.hstack(  ( X, year ) )

In [66]:
import numpy as np
np.hstack( (X,X ) )

array([<95x374 sparse matrix of type '<class 'numpy.float64'>'
	with 768 stored elements in Compressed Sparse Row format>,
       <95x374 sparse matrix of type '<class 'numpy.float64'>'
	with 768 stored elements in Compressed Sparse Row format>], dtype=object)

In [74]:
X1=np.random.rand(95,1)

In [75]:
import scipy
scipy.sparse.hstack(  ( X,X1 ) )

<95x375 sparse matrix of type '<class 'numpy.float64'>'
	with 863 stored elements in COOrdinate format>

In [76]:
scipy.sparse.hstack(  ( X,X1 ) )

<95x375 sparse matrix of type '<class 'numpy.float64'>'
	with 863 stored elements in COOrdinate format>

In [ ]:
https://stackoverflow.com/questions/32281529/what-is-the-correct-way-to-mix-feature-sparse-matrices-with-sklearn

In [56]:
for Eurolex_json in Eurolex_jsons:
    print( json.dumps( Eurolex_json[0] , indent=4 ) )
    print( "\n\n\n\n\n\n\n\n"  )

{
    "id": "0469cda9-a1db-5a84-9e16-d858d28198df",
    "title": [
        "A framework for measuring and managing liquidity "
    ],
    "status": [
        "Superseded"
    ],
    "type": [
        "Guidelines"
    ],
    "date": [
        "1992-09-28T00:00:00Z"
    ],
    "url": [
        "https://www.bis.org/publ/bcbs10b.htm"
    ],
    "website": [
        "BIS"
    ],
    "summary": [
        "Note: This document has been superseded by Sound Practices for Managing Liquidity in Banking Organisations (Basel Committee Publications No. 69, February 2000)In its work on the supervision of liquidity, the Basle Committee has focused on developing a greater understanding of the way in which international banks manage their liquidity on a global basis, on the premise that supervision of liquidity is particularly effective if based on a dialogue between bank and supervisor. This paper brings together practice and techniques currently employed by major international banks in a single analyti

In [ ]:
print(json.dumps(cars, indent = 4))

In [48]:
for key in Eurolex_jsons[10][0]:
    print (key )
    print(  Eurolex_jsons[10][0][key]  )

id
3e505226-2365-5d50-92c5-a5d4e2335d14
title
['Standards and Processes for Global Securities Financing Data Collection and Aggregation']
date
['2015-11-18T00:00:00Z']
url
['https://www.fsb.org/2015/11/standards-and-processes-for-global-securities-financing-data-collection-and-aggregation-3/']
website
['FSB']
summary
['Securities financing transactions (SFTs) such as securities lending and repurchase agreements (repos) play a crucial role in supporting price discovery and secondary market liquidity for a wide variety of securities. However, these\xa0transactions can also be used to take on leverage as well as mismatched maturity and liquidity exposures. This report builds on policy recommendations to address financial stability risks in SFTs published in August 2013, in particular its recommendations to improve transparency of securities financing markets, and on the public consultation findings. The report sets out finalised standards and processes to allow the FSB to periodically col

In [36]:
Eurolex_json[0]

{'id': '0469cda9-a1db-5a84-9e16-d858d28198df',
 'title': ['A framework for measuring and managing liquidity '],
 'status': ['Superseded'],
 'type': ['Guidelines'],
 'date': ['1992-09-28T00:00:00Z'],
 'url': ['https://www.bis.org/publ/bcbs10b.htm'],
 'website': ['BIS'],
 'summary': ['Note: This document has been superseded by Sound Practices for Managing Liquidity in Banking Organisations (Basel Committee Publications No. 69, February 2000)In its work on the supervision of liquidity, the Basle Committee has focused on developing a greater understanding of the way in which international banks manage their liquidity on a global basis, on the premise that supervision of liquidity is particularly effective if based on a dialogue between bank and supervisor. This paper brings together practice and techniques currently employed by major international banks in a single analytical framework. While no bank may follow this framework precisely, it is a model which is believed to be an example of c

In [35]:
Eurolex_json[0]['summary']

['Note: This document has been superseded by Sound Practices for Managing Liquidity in Banking Organisations (Basel Committee Publications No. 69, February 2000)In its work on the supervision of liquidity, the Basle Committee has focused on developing a greater understanding of the way in which international banks manage their liquidity on a global basis, on the premise that supervision of liquidity is particularly effective if based on a dialogue between bank and supervisor. This paper brings together practice and techniques currently employed by major international banks in a single analytical framework. While no bank may follow this framework precisely, it is a model which is believed to be an example of current sound management practice and which could provide useful guidance to all banks.']

In [25]:
Eurolex_json[1]

{'attr_document_id': ['0469cda9-a1db-5a84-9e16-d858d28198df'],
 'id': '8a09b11b-4689-57d8-aa37-ad2caec92c76',
 'attr_url': ['https://www.bis.org/publ/bcbs10b.pdf'],
 'attr_date': ['2012-02-20T15:21:52Z'],
 'attr_pdf_pdfversion': ['1.6'],
 'attr_pdf_docinfo_title': ['A framework for measuring and managing liquidity - Basel Committee - September 1992'],
 'attr_xmp_creatortool': ['Acrobat PDFMaker 6.0 for Word'],
 'attr_company': ['BIS-BRI-BIZ'],
 'attr_access_permission_can_print_degraded': ['true'],
 'attr_subject': ['Full text of Basel Committee publication "A framework for measuring and managing liquidity"'],
 'attr_dc_format': ['application/pdf; version=1.6'],
 'attr_pdf_docinfo_creator_tool': ['Acrobat PDFMaker 6.0 for Word'],
 'attr_access_permission_fill_in_form': ['true'],
 'attr_stream_name': ['bcbs10b.pdf'],
 'attr_pdf_encrypted': ['false'],
 'attr_dc_title': ['A framework for measuring and managing liquidity - Basel Committee - September 1992'],
 'attr_modified': ['2012-02-20T